In [5]:
using Primes

n = BigInt(25851029405939727929221562380823);

In [3]:
factores_pot_2 = [2^ex for ex in 1:factor(n+1)[2]]

3-element Array{Int64,1}:
 2
 4
 8

El único cofactor que puede ser impar es el del $8$.

In [6]:
n1 = div(n+1, 8)

3231378675742465991152695297603

# Apartado 1

### Pasa el test de Miller-Rabin para varias bases y si fuera necesario el de Solovay-Strassen para ver si n1 es compuesto.

In [7]:
function miller_rabin_rec(a, acu, n, var_name="n")
    if acu % 2 == 0
        # Si n es par aplicamos Miller-Rabin a n/2, de esta forma delegamos a la llamada que calcula el primer elemento
        # de la sucesión el cálculo del mínimo m tal que n = 2^k * m.
        # También delegamos a esta llamada la información sobre la iteración en la que estamos.
        i, m = miller_rabin_rec(a, BigInt(acu/2), n, var_name)
    else
        # Si n no es impar se establecen los valores de la primera iteración
        i = 0 # Número de iteración
        m = acu # Mínimo m tal que n - 1 = 2^k * m
    end
    
    println("$a^$(2^i)m = $(powermod(a, (2^i)*m, n)) mod $var_name")
    return i + 1, m
end


function miller_rabin(a, n; var_name="n")
    # Llamamos a la fución con n-1 para que se cumpla n - 1 = 2^k * m
    return miller_rabin_rec(a, n-1, n, var_name)
    end;

In [8]:
miller_rabin(2, n1, var_name="n1");

2^1m = 1485140714446976422519866021041 mod n1
2^2m = 932835830145814621440467477605 mod n1


Basta con fijarse en que la sucesión para $a=2$ no termina en $1$, por tanto $2$ es un certificado de composición para $n1$.

# Apartado 2

### En caso que hayas encontrado un certificado de composición para n1, aplica el método $\rho$ de Pollard y/o el cfrac para encontrar factores de ambos.

In [9]:
function pollard(n, t = 100, f = x -> x^2 + 1)
    x = 1
    y = x
    i = 0
    
    #println("Paso | x | y | mcd")
    #println("$i | $x | $y | -")
    
    while i < t
        i += 1
        x = f(x) % n
        y = f(f(y)) % n
        g = gcd(x - y, n)
        
        #println("$i | $x | $y | $g")
        
        if 1 < g && g < n
            return g, div(n, g)
        end
    end
    
    println("No hay divisores con $t iteraciones.")
end;

In [10]:
pollard(n1)

(3, 1077126225247488663717565099201)

# Apartado 3

### Para factores primos menores que 10.000 puedes usar la lista de la página web para certificar su primalidad. Si los factores son mayores y no has demostrado que sean compuestos, encuentra una s.L. para certificar su primalidad.

Descomponemos $n1$, el cual sabemos que es compuesto.

In [11]:
f1, f2 = pollard(n1)

(3, 1077126225247488663717565099201)

Dado que $f_1=3$ es primo, intentamos descomponer $f_2$.

In [48]:
pollard(f2)

No hay divisores con 100 iteraciones.


Dado que no se ha podido descomponer comprobamos si es un posible primo.

In [33]:
miller_rabin(2, f2);

2^1m = 582602755878034070823404469643 mod n1
2^2m = 668259076182893834409451532818 mod n1
2^4m = 838818130839183969075635682806 mod n1
2^8m = 326668062855906550766650306145 mod n1
2^16m = 293945926148165482217089327868 mod n1
2^32m = 294464354113976351607079186896 mod n1
2^64m = 700810738728178350210920298809 mod n1


El test de Miller-Rabin nos dice que es compuesto, por tanto lo que tenemos que hacer es darle suficientes iteraciones al algoritmo $\rho$ de Pollard.

In [12]:
f21, f22 = pollard(f2, 10000)

(12444073, 86557369540301528584537)

Ahora vemos si $f_{21}$ es compuesto mediante Miller-Rabin.

In [42]:
for a in [2, 3, 5, 7, 11]
    println("Miller-Rabin para a = $a:\n")
    i, m = miller_rabin(a, f21, var_name="f21")
    println("\nm = $m\n\n")
end

Miller-Rabin para a = 2:

2^1m = 12444072 mod f21
2^2m = 1 mod f21
2^4m = 1 mod f21
2^8m = 1 mod f21

m = 1555509


Miller-Rabin para a = 3:

3^1m = 12444072 mod f21
3^2m = 1 mod f21
3^4m = 1 mod f21
3^8m = 1 mod f21

m = 1555509


Miller-Rabin para a = 5:

5^1m = 2134970 mod f21
5^2m = 7178022 mod f21
5^4m = 12444072 mod f21
5^8m = 1 mod f21

m = 1555509


Miller-Rabin para a = 7:

7^1m = 1826087 mod f21
7^2m = 5266051 mod f21
7^4m = 12444072 mod f21
7^8m = 1 mod f21

m = 1555509


Miller-Rabin para a = 11:

11^1m = 7178022 mod f21
11^2m = 12444072 mod f21
11^4m = 1 mod f21
11^8m = 1 mod f21

m = 1555509




Teninedo en cuenta que $12444072 \equiv_{f_{21}} -1$, $f_{21}$ pasa el test de Miller-Rabin para cinco bases y es muy probable que sea primo. Buscamos un Lucas-Certificado.

Primero factorizamos $f_{21}+1$.

In [56]:
pollard(f21+1)

(2, 6222037)

In [52]:
pollard(6222037)

(29, 214553)

In [60]:
pollard(214553)

(41, 5233)

Por tanto $f_{21}+1 = 2 \cdot 29 \cdot 41 \cdot 5233$

Intentamos encontrar un valor de $Q$ para el cual cada término $U_e$ de la sucesión de Lucas sea distinto de $0$, con $e = \frac{n+1}{d}$ y $d$ un factor de $n+1$.

In [34]:
function lucas(n_, mod)
    n = string(n_; base=2)
    (U0, U1) = (0, 1)
    for e in n
        if e == '0'
            (U0, U1) = (2*U0*U1 - P*U0^2, U1^2 - Q*U0^2) .% mod
            if U0 < 0 U0 = mod + U0 end
            if U1 < 0 U1 = mod + U1 end
        elseif e == '1'
            (U0, U1) = (U1^2 - Q*U0^2, P*U1^2 - 2*Q*U0*U1) .% mod
            if U0 < 0 U0 = mod + U0 end
            if U1 < 0 U1 = mod + U1 end
        end
    end
    V = (2*U1 - P*U0) % mod
    (U0, V)
    end;

In [44]:
P = 1
Q = 2 
while true
    lucas(f21+1, f21)[1] == 0 &&
    lucas(div(f21+1, 2), f21)[1] != 0 &&
    lucas(div(f21+1, 29), f21)[1] != 0 &&
    lucas(div(f21+1, 41), f21)[1] != 0 &&
    lucas(div(f21+1, 5233), f21)[1] != 0 &&
    return Q
    Q += 1
end

10

Vemos que efectivamente con $P=1$, $Q=10$ se cumple esto

In [45]:
println("U_{f21+1} = $(lucas(f21+1, f21)[1])")
println("U_{(f21+1)/2} = $(lucas(div(f21+1, 2), f21)[1])")
println("U_{(f21+1)/29} = $(lucas(div(f21+1, 29), f21)[1])")
println("U_{(f21+1)/41} = $(lucas(div(f21+1, 41), f21)[1])")
println("U_{(f21+1)/5233} = $(lucas(div(f21+1, 5233), f21)[1])")

U_{f21+1} = 0
U_{(f21+1)/2} = 11434288
U_{(f21+1)/29} = 11515628
U_{(f21+1)/41} = 7867086
U_{(f21+1)/5233} = 7040827


Certificamos así que $f_{21} = 12444073$ es primo.

Solo nos queda analizar $f_{22}$. Comenzaremos pasándole el test de Miller-Rabin.

In [47]:
for a in [2, 3, 5, 7, 11]
    println("Miller-Rabin para a = $a:\n")
    i, m = miller_rabin(a, f22, var_name="f22")
    println("\nm = $m\n\n")
end

Miller-Rabin para a = 2:

2^1m = 86557369540301528584536 mod f22
2^2m = 1 mod f22
2^4m = 1 mod f22
2^8m = 1 mod f22

m = 10819671192537691073067


Miller-Rabin para a = 3:

3^1m = 1 mod f22
3^2m = 1 mod f22
3^4m = 1 mod f22
3^8m = 1 mod f22

m = 10819671192537691073067


Miller-Rabin para a = 5:

5^1m = 9336927384376328826521 mod f22
5^2m = 60232763493631174304738 mod f22
5^4m = 86557369540301528584536 mod f22
5^8m = 1 mod f22

m = 10819671192537691073067


Miller-Rabin para a = 7:

7^1m = 9336929926571709100219 mod f22
7^2m = 26324606046670354279799 mod f22
7^4m = 86557369540301528584536 mod f22
7^8m = 1 mod f22

m = 10819671192537691073067


Miller-Rabin para a = 11:

11^1m = 9336929926571709100219 mod f22
11^2m = 26324606046670354279799 mod f22
11^4m = 86557369540301528584536 mod f22
11^8m = 1 mod f22

m = 10819671192537691073067




Teninedo en cuenta que $86557369540301528584536 \equiv_{f_{22}} -1$, $f_{22}$ pasa el test de Miller-Rabin para cinco bases y es muy probable que sea primo. Buscamos un Lucas-Certificado.

Primero factorizamos $f_{22} + 1$.

In [49]:
pollard(f22+1)

(2, 43278684770150764292269)

In [51]:
pollard(43278684770150764292269, 1000)

(227, 190654998987448300847)

In [57]:
pollard(190654998987448300847, 100000000)

(349411, 545646814174277)

In [71]:
pollard(349411, 100000000)

No hay divisores con 100000000 iteraciones.


In [67]:
pollard(545646814174277, 1000000)

(2786587, 195811871)

In [74]:
pollard(2786587, 100000000)

No hay divisores con 100000000 iteraciones.


In [75]:
pollard(195811871, 100000000)

No hay divisores con 100000000 iteraciones.


Por acortar el ejercicio veamos que $349411$, $2786587$ y $195811871$ son primos mediante una función externa. El método para comprobarlo, sin embargo, sería el mismo que estamos aplicando a $f_{22}$, i.e. factorizamos $n+1$ y buscamos unos $P$ y $Q$ para que los términos $U_{\frac{n+1}{d}}$, con d un factor de $n+1$, sean distintos de 0.

In [76]:
isprime(349411) && isprime(2786587) && isprime(195811871)

true

Por tanto tenemos que $f_{22}+1 = 2 \cdot 227 \cdot 349411 \cdot 2786587 \cdot 195811871$

Obtenemos ahora un Lucas-Certificado.

In [78]:
P = 1
Q = 2 
while true
    lucas(f22+1, f22)[1] == 0 &&
    lucas(div(f22+1, 2), f22)[1] != 0 &&
    lucas(div(f22+1, 227), f22)[1] != 0 &&
    lucas(div(f22+1, 349411), f22)[1] != 0 &&
    lucas(div(f22+1, 2786587), f22)[1] != 0 &&
    lucas(div(f22+1, 195811871), f22)[1] != 0 &&
    return Q
    Q += 1
end

20

Tenemos que con $P = 1$, $Q = 20$ se verifica que $f_{22} = 86557369540301528584537$ es primo.

Finalmente tenemos que $n1 = 3 \cdot 12444073 \cdot 86557369540301528584537$

# Apartado 4

### Aplica lo anterior hasta encontrar la factorización en primos de n+1.

Como $n1 = \frac{n+1}{8}$, se tiene que $n+1 = 8n1 = 2^3 \cdot 3 \cdot 12444073 \cdot 86557369540301528584537$

Comprobamos que el resultado efectivamente es correcto.

In [81]:
factor(n+1)

2^3 ⋅ 3 ⋅ 12444073 ⋅ 86557369540301528584537

# Apartado 5

### Encuentra una s.L. para certificar la primalidad de n.

Conociendo ya los factores de $n+1$ todo lo que queda es encontrar el Lucas-Certificado.

In [82]:
P = 1
Q = 2 
while true
    lucas(n+1, n)[1] == 0 &&
    lucas(div(n+1, 2), n)[1] != 0 &&
    lucas(div(n+1, 3), n)[1] != 0 &&
    lucas(div(n+1, 12444073), n)[1] != 0 &&
    lucas(div(n+1, 86557369540301528584537), n)[1] != 0 &&
    return Q
    Q += 1
end

7

Por tanto con $P=1$, $Q=7$ demostramos que $n$ es primo.